# 1. Vector Space Model of Word Meaning

The goal of this problem set is to make you familiar with vector space model of word meaning. You may reuse some of functions you coded in Assignment 1. 

### Warning: This assignment may take substantial time to run if you are not optimizing your code. Make sure you have plenty of time to run if you are new to programming.

Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with current events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not. 



Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


When you run the `ls` command below, you should see the files in the tweets folder.

> Indented block






In [ ]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
covid-tweets-2020-08-10-2020-08-21.trigrams.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


In [ ]:
# let's read tweets. These tweets are already tokenized and cleaned (Assignment 1)
tweets = open("/content/drive/My Drive/tweets/covid-tweets-2020-08-10-2020-08-21.tokenized.txt", "r").read().split("\n")
tweets = [tweet.split() for tweet in tweets]

print(len(tweets))


312878


## Problem 1.1: Word space model

Compute the most important context words of `ventilator`. Use Pointwise Mutual Information (PMI) to rank the context words (Refer to Lecture 4).

We define context as up to 3 words to the left and 3 words to the right. Ignore stop words and words that do not start with [a-z#]. Also ignore words that are not in the top 1000 frequent words.

These context words define the dimensions of the vector space model. Represent each word as a vector (dictionary/counter) of context words with PMI as the importance of the context word. Print the top 20 context words for each.

This is the sample output I got for `ventilator`. Your numbers need not match mine but the ranked list should look close to what I have.

```
[('put', 18.280538283196606), ('wearing', 17.587373569812726), ('even', 17.58651933524197), ('like', 17.402738298715878), ('covid', 17.172590097063086), ('patients', 16.894419647496004), ('use', 16.894298589380956), ('die', 16.89426559608771), ('days', 16.89415252713107), ('needed', 16.489137134110106), ('month', 16.48907033839664), ('weeks', 16.488913820220848), ('away', 16.48879303327717), ('week', 16.488739054051933), ('person', 16.488678720881293), ('good', 16.488160838026904), ('deaths', 16.487822204799755), ('go', 16.487564042558112), ('would', 16.48707075078768), ('one', 16.48706217686235), ('get', 16.486565870239033)]
```



Let's first load stop words.

In [ ]:
stop_words = set()
def load_stop_words():
  words = open("/content/drive/My Drive/tweets/stop_words.txt", "r").read().split("\n")
  for word in words:
    stop_words.add(word.strip())

load_stop_words()
print(len(stop_words))
print(stop_words)

134
{'if', 'on', 'did', 'here', 'can', 'myself', 'down', 'above', 'doing', 'all', 'and', 'from', 'any', 'am', 'where', 'until', 'some', 'over', 'now', 'don', 'do', 'we', 'with', 'who', 'this', 'after', 'of', 'were', 'itself', 'but', 'been', 'he', 'so', '@USER', 'me', 'by', 'while', 'once', 'again', 'being', 'is', 'not', 'during', 'ourselves', 'that', 'only', 'same', 'ours', 'will', 'both', 'its', 'when', 'whom', 'she', 'how', 'there', 'was', 's', 'should', 'those', "'ve", 'does', 'in', 'it', 'themselves', 'i', 'what', 'his', 'between', 'nor', 'before', 'to', 'a', 'amp', 'yours', 'very', 'each', 'into', 'herself', 'their', 'most', 'are', 'out', 'up', 'the', 't', "n't", 'why', 'yourselves', 'her', 'under', "'ll", 'be', 'other', 'such', 'him', 'then', 'have', 'because', 'no', 'had', 'below', 'further', "'ld", 'off', 'more', 'than', 'yourself', 'himself', 'which', 'an', 'at', 'own', 'you', 'few', 'our', 'my', 'your', 'has', 'about', 'hers', 'against', 'having', 'these', 'theirs', 'for', 't



```
# This is formatted as code
```

Let's build the word vectors

In [ ]:
from collections import Counter
from itertools import combinations
from math import log
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pformat
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, norm
from nltk.util import skipgrams
from string import punctuation
from scipy import sparse
from scipy.sparse import linalg
import re
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()
stemmer= PorterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
unigram_counts = Counter()
skipgram_counts = Counter()
x2i, i2x = {}, {}
pmi_samples = Counter()

tweets_trimmed = [[token for token in tweet if token not in stop_words and (re.match(r"[a-z#]+", token[0]))] for tweet in tweets]

In [ ]:
def compute_unigrams():
  global unigram_counts
  for idx, tweet in enumerate(tweets_trimmed):
    for token in tweet:
      unigram_counts[token] += 1
    if idx % 50000 == 0:
        print(f'finished {idx/len(tweets_trimmed):.2%} of tweets')

  # # Remove all unigrams in in top 1000
  # most_common_unigram = unigram_counts.most_common(1000)
  # for k,v in list(unigram_counts.copy().items()):
  #   if (k,v) not in most_common_unigram:
  #     del unigram_counts[k]
  print('Most common unigram: {}'.format(unigram_counts.most_common(10)))
  print('Trimmed vocab size should be 1000: {}'.format(len(unigram_counts)))

compute_unigrams()

finished 0.00% of tweets
finished 15.98% of tweets
finished 31.96% of tweets
finished 47.94% of tweets
finished 63.92% of tweets
finished 79.90% of tweets
finished 95.88% of tweets
Most common unigram: [('covid', 71281), ('pandemic', 50557), ('covid-19', 33591), ('people', 32087), ('n’t', 31053), ('mask', 28407), ('get', 21681), ('like', 21043), ('coronavirus', 19949), ('trump', 19889)]
Trimmed vocab size should be 1000: 132037


In [ ]:
def compute_skip_grams():
  global skipgram_counts
  for idx, tweet in enumerate(tweets_trimmed):
      # 3 words left and right
      skip_gram = list(skipgrams(tweet, 2, 3))
      for context in skip_gram:
        skipgram_counts[context] += 1
      if idx % 50000 == 0:
        print(f'finished {idx/len(tweets_trimmed):.2%} of tweets')
  
  # Remove infrequent bigrams (not in unigrams)
  # for x, y in list(skipgram_counts.keys()):
  #   if x not in unigram_counts or y not in unigram_counts:
  #     del skipgram_counts[(x,y)]

  print('Skipgram size: {}'.format(len(skipgram_counts)))
  print('Most common skipgrams: {}'.format(skipgram_counts.most_common(10)))


compute_skip_grams()

finished 0.00% of tweets
finished 15.98% of tweets
finished 31.96% of tweets
finished 47.94% of tweets
finished 63.92% of tweets
finished 79.90% of tweets
finished 95.88% of tweets
Skipgram size: 5711825
Most common skipgrams: [(('wear', 'mask'), 13493), (('social', 'distancing'), 6776), (('stay', 'home'), 6044), (('ca', 'n’t'), 4990), (('new', 'case'), 4267), (('covid', 'death'), 3119), (('case', 'death'), 3104), (('wearing', 'mask'), 3031), (('gon', 'na'), 2887), (('get', 'covid'), 2602)]


In [ ]:
def build_unigram_index_lookup():
  global x2i
  global i2x
  for i, x in enumerate(unigram_counts.keys()):
    x2i[x] = i
    i2x[i] = x

build_unigram_index_lookup()

In [ ]:
# THIS CODE IS SHITAAAAZZZZZ
# help here: https://www.kaggle.com/gabrielaltay/word-vectors-from-pmi-matrix

sxy = 0
sx = 0

def PMI(word1, word2, n):
  # You have to store frequencies of individual words and (word, context word) 
  # pairs to compute this. You can compute them beforehand in order to avoid 
  # counting every time when this function is called.
  # Write your code
  global skipgram_counts
  global unigram_counts
  global sx
  global sxy
  # nxy = skipgram_counts[(word1, word2)]
  # N = len(tweets_trimmed)
  # nx = unigram_counts[word1]
  # ny = unigram_counts[word2]
  # pmi = log((nxy * N) / (nx * ny))
  return (log((n / sxy) / (unigram_counts[word1] / sx) / (unigram_counts[word2] / sx)))


def build_word_vector(word):
  # Write your code
  global skipgram_counts
  global unigram_counts
  global x2i
  global i2x
  global sx
  global sxy
  sx = sum(unigram_counts.values())
  sxy = sum(skipgram_counts.values())
  data, rows, cols = [], [], []
  pmi_sample = Counter()

  idx = 0
  for (x, y), n in skipgram_counts.items():
    rows.append(x2i[x])
    cols.append(x2i[y])
    data.append(PMI(x,y, n))
    pmi_sample[(x,y)] = data[-1]
    
    if idx % 500000 == 0:
        print(f'finished {idx/len(skipgram_counts):.2%} of tweets')
    idx += 1



  PMI_matrix = csc_matrix((data, (rows, cols)))
  ans = Counter()
  print(pmi_sample.most_common()[:20])
  for (x,y), n in pmi_sample.items():
    if (x == word or y == word):
      ans[(x,y)] = n
  print(ans.most_common(5))

def print_top_dimensions(word_vector, n):
  # print top n dimensions sorted in the order of importance.
  ...

v1 = build_word_vector('ventilator')

# print_top_dimensions(v1, 20) # print top 20 dimensions along with their weights


finished 0.00% of tweets
finished 8.75% of tweets
finished 17.51% of tweets
finished 26.26% of tweets
finished 35.02% of tweets
finished 43.77% of tweets
finished 52.52% of tweets
finished 61.28% of tweets
finished 70.03% of tweets
finished 78.78% of tweets
finished 87.54% of tweets
finished 96.29% of tweets
[(('#julitonosecuelgue', '#regimennopodraconad'), 14.035149816323184), (('#julitonosecuelgue', '#losdanieles'), 14.035149816323184), (('#julitonosecuelgue', '#formulae'), 14.035149816323184), (('#regimennopodraconad', '#losdanieles'), 14.035149816323184), (('#regimennopodraconad', '#formulae'), 14.035149816323184), (('#losdanieles', '#formulae'), 14.035149816323184), (('coms', 'horas'), 14.035149816323184), (('#mrmojorisin', '#lionsinthestreet'), 14.035149816323184), (('#mrmojorisin', '#dawnshighway'), 14.035149816323184), (('#mrmojorisin', '#summersalmostgone'), 14.035149816323184), (('#lionsinthestreet', '#dawnshighway'), 14.035149816323184), (('#lionsinthestreet', '#summersalmos

## Problem 1.2: Compute the word similarity between words using Cosine Similarity.

Compute cosine similarity between the following pair of words: 
```
('ventilator', 'covid-19')
('ventilator', 'lockdown')
('ventilator', 'mask')
('ventilator', 'ppe')
```

Outputs of my code are:
```
('ventilator', 'covid-19') 0.17076006036635358
('ventilator', 'lockdown') ...
('ventilator', 'mask') 0.19229601085517933
('ventilator', 'ppe') ...
```


In [ ]:
def cosine_similarity(vector1, vector2):
  v2 = list(vector1.values())
  v2 = list(vector2.values())

  dot_prod = np.dot(v1, v2)
  mag = LA.norm(v1) * LA.norm(v2)
  return dot_prod / mag

ventilator = build_word_vector('ventilator')
covid19 = build_word_vector('covid-19')
lockdown = build_word_vector('lockdown')
mask = build_word_vector('mask')
ppe = build_word_vector('ppe')

print(('ventilator', 'covid-19'), cosine_similarity(ventilator, covid19))
print(('ventilator', 'lockdown'), cosine_similarity(ventilator, lockdown))
print(('ventilator', 'mask'), cosine_similarity(ventilator, mask))
print(('ventilator', 'ppe'), cosine_similarity(ventilator, ppe))

finished 0.00% of tweets
finished 8.75% of tweets
finished 17.51% of tweets
finished 26.26% of tweets
finished 35.02% of tweets
finished 43.77% of tweets
finished 52.52% of tweets
finished 61.28% of tweets
finished 70.03% of tweets
finished 78.78% of tweets
finished 87.54% of tweets
finished 96.29% of tweets
[(('#julitonosecuelgue', '#regimennopodraconad'), 14.035149816323184), (('#julitonosecuelgue', '#losdanieles'), 14.035149816323184), (('#julitonosecuelgue', '#formulae'), 14.035149816323184), (('#regimennopodraconad', '#losdanieles'), 14.035149816323184), (('#regimennopodraconad', '#formulae'), 14.035149816323184), (('#losdanieles', '#formulae'), 14.035149816323184), (('coms', 'horas'), 14.035149816323184), (('#mrmojorisin', '#lionsinthestreet'), 14.035149816323184), (('#mrmojorisin', '#dawnshighway'), 14.035149816323184), (('#mrmojorisin', '#summersalmostgone'), 14.035149816323184), (('#lionsinthestreet', '#dawnshighway'), 14.035149816323184), (('#lionsinthestreet', '#summersalmos

AttributeError: ignored

## Problem 1.3: What can you tell about these words from the similarities?

1. `ventilator` when compared with `covid-19, lockdown, mask, ppe`
2. `pandemic` when compared with `covid-19, lockdown, mask, ppe`
3. `president` compared with `trump, biden`
4. `trudeau` compared with `trump, biden`



1. They similar
2. They similar
3. They similar
4. They similar

# Let's play with word2vec

First let's load word2vec. I am using [gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) but feel free to use any libraries or tools.

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from gensim.models import KeyedVectors
EMBEDDING_FILE = '/content/drive/My Drive/tweets/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

## Problem 1.4: Compute the top 5 similar words of `ventilator` using word2vec?

---



In [ ]:
# Write your code here
word2vec.most_similar(['ventilator'] , topn = 5)

# Problem 1.5: Word analogy

If I told you the plural of `car` is `cars`, can you automatically find the plural of `hypothesis` and `man` using word2vec.

Similarly, if I told you a newborn `dog` is called `puppy`, can you automatically find what are the newborn words of `cat` and `sheep` using word2vec?


In [ ]:
word2vec.most_similar(positive=['hypothesis','cars'], negative=['car'] )[0][0]
word2vec.most_similar(positive=['man','cars'], negative=['car'] )[0][0]

word2vec.most_similar(positive=['cat','puppy'], negative=['dog'] )[0][0]
word2vec.most_similar(positive=['sheep','puppy'], negative=['dog'] )[0][0]

# 2. Topic Models 

The goal of this part is to make you familiar with topic models. You may reuse some of functions you coded for the previous assignments.

## Data Download and Setup

Let us start by downloading the news section of the Brown corpus:

In [ ]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
documents = [brown.words(fileid) for fileid in brown.fileids()]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


Let us inspect some of the documents:

In [ ]:
print("The news section of the Brown corpus contains {} documents.".format(len(documents)))
for i in range(3):
  document = documents[i]
  print("Document {} has {} words: {}".format(i, len(document), document))

The news section of the Brown corpus contains 500 documents.
Document 0 has 2242 words: ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]
Document 1 has 2277 words: ['Austin', ',', 'Texas', '--', 'Committee', 'approval', ...]
Document 2 has 2275 words: ['Several', 'defendants', 'in', 'the', 'Summerdale', ...]


Finally, let us download a list of stopwords for later.

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
print(stopwords_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

## Problem 2.1: Document-Term Matrix

Create a document-term matrix with tf-idf. You should preprocess documents by: 1) lowercasing words, 2) excluding stopwords, and 3) including alphanumeric strings only.

```python
import numpy as np
def create_tfidf_matrix(documents: List[List[str]]) -> (np.array, List[str]):
  # Args:
  #   documents: list of documents, each document being a list of words.
  # Outputs:
  #   tfidf_matrix: np.array of shape (num_documents, vocabulary_size)
  #   vocabulary: a list of terms corresponding to the columns of the matrix.

tfidf_matrix, vocabulary = create_tfidf_matrix(documents)
```

How sparse is this matrix? Calculate the ratio between cells with value 0 and the total number of cells. 


In [33]:
import re
from typing import List

def preprocess(documents: List[List[str]]):
  return_docs = []
  documents_processed = [[token.lower() for token in document if token not in stopwords_list and token.isalnum()] for document in documents]
  return documents_processed
  # for doc in documents_processed:
  #   return_docs.append(' '.join(doc))


documents_processed = preprocess(documents)

In [34]:
def compute_word_set(documents):
  wordset = set()
  for document in documents:
    wordset = wordset.union(set(document))
  return wordset

wordset = compute_word_set(documents)

In [35]:
def compute_word_dict(documents):
  word_dict_list = []
  for document in documents:
    word_dict = dict.fromkeys(wordset, 0)
    for word in document:
      word_dict[word] += 1
    word_dict_list.append(word_dict)
  return word_dict_list

word_dict_list = compute_word_dict(documents)


In [36]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

tf = []
for idx, doc in enumerate(documents):
  tf.append(computeTF(word_dict_list[idx], doc))



In [37]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

idfs = computeIDF(tf)

In [38]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

tfidf = []
for i in tf:
  tfidf.append(computeTFIDF(i, idfs))

In [39]:
import pandas as pd
import numpy as np

def create_tfidf_matrix(documents: List[List[str]]) -> (np.array, List[str]):
  # Args:
  #   documents: list of documents, each document being a list of words.
  # Outputs:
  #   tfidf_matrix: np.array of shape (num_documents, vocabulary_size)
  #   vocabulary: a list of terms corresponding to the columns of the matrix.

  global tfidf
  tfidfDF = pd.DataFrame(tfidf)
  tfidf_matrix = pd.DataFrame(tfidfDF).to_numpy()
  voc = list(tfidfDF.columns.values)
  return (tfidf_matrix, voc)

tfidf_matrix, vocabulary =  create_tfidf_matrix(documents_processed)
sparsity = 1.0 - ( count_nonzero(tfidf_matrix) / float(tfidf_matrix.size) )
print("Sparcity of a matrix is {}".format(sparsity))

Sparcity of a matrix is 0.9858975328683305


## Problem 2.2: Latent Semantic Analysis

We perform LSA to obtain document embeddings `U` and term embeddings `VT`.

```python
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(tfidf_matrix, 
                              n_components=10,
                              n_iter=100,
                              random_state=42)
```

Define a function to find the 5 most relevant terms for each of the 10 latent dimensions (tip: you should make use of VT and the vocabulary).

```python
def extract_salient_words(VT: np.array, 
                  vocabulary: List[str]
                  ) -> salient_words: dict[int, List[str]]:
  # Args:
  #  VT: a numpy array of size (n_components, vocabulary_size)
  #  vocabulary: a list of words of size vocabulary_size
  # Outputs:
  #  salient_words: a dictionary with the latent dimension indices as keys and a list of its 5 most salient words as values

salient_words = extract_salient_words(VT, vocabulary)

for key, value in salient_words:
  print("Concept {}: {}".format(str(key), " ".join(value)))
```

In [ ]:
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(tfidf_matrix, 
                              n_components=10,
                              n_iter=100,
                              random_state=42)

NameError: ignored

In [ ]:
from typing import List, Dict
import numpy as np

def extract_salient_words(VT: np.array, vocabulary: List[str]) -> Dict[int, List[str]]:
  # Args:
  #  VT: a numpy array of size (n_components, vocabulary_size)
  #  vocabulary: a list of words of size vocabulary_size
  # Outputs:
  #  salient_words: a dictionary with the latent dimension indices as keys and a list of its 5 most salient words as values
  salient_words: Dict[int, List[str]] = {}
  for i in range(len(VT)):
    tfidf_sorting = np.argsort(VT[i]).flatten()[::-1]
    n = 5
    voc = np.array(vocabulary)
    top_n = voc[tfidf_sorting]
    salient_words[i] = top_n[:n].tolist()
  return salient_words

salient_words = extract_salient_words(VT, vocabulary)
for key, value in salient_words.items():
  print("Concept {}: {}".format(str(key), " ".join(value)))

Concept 0: af polynomial operator theorem q
Concept 1: af polynomial operator theorem diagonalizable
Concept 2: anode holder arc temperature plug
Concept 3: anode phil mike deegan jess
Concept 4: staining nonspecific clover sections wtv
Concept 5: dictionary text cell platform information
Concept 6: platform gyro accelerometer leveling axis
Concept 7: q tangent secants c curve
Concept 8: feed phil milligrams cattle grams
Concept 9: phil mike deegan eddie anniston


## Problem 2.3: Document Retrieval

Given a text query, view this as a mini document, and compare it to your documents in the low-dimensional space.

First, we need to map a query into a pseudo-document embedding.
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/ed5d0397ee6b44f72f77743029d3943932118fa2" alt="Query" height="35"/>

Then, you will need to implement a function to calculate the cosine similarity between this embedded query and all the document embeddings.

Retrieve the indices of the top-3 documents with the highest cosine similarity with the following queries:


```python
query1 = ['T.', 'V.', 'Barker', 'developed', 'the', 'classification-angle', 'system']
query2 = ['imitation', 'vs.', 'formalism' 'in', 'philosophical', 'debates']
query3 = ['Krim', 'attended', 'the', 'University', 'of', 'North', 'Carolina', 'to', 'follow', 'Thomas', 'Wolfe']
```



## Problem 2.4: Document Clustering

```python
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

num_clusters = 10
km = KMeans(n_clusters=num_clusters)
document_embeddings = U * Sigma
km.fit(document_embeddings)
clusters = km.labels_.tolist()
print(clusters)
```

Let us now plot the document embeddings and their clusters:

```python
import umap
embedding = umap.UMAP(n_neighbors=100, min_dist=0.5, random_state=42).fit_transform(document_embeddings)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], c=clusters, s=20, edgecolor='none')
plt.show()
```

What are the differences you observe by using a different number of `n_components` in LSA or `n_clusters` in K-Means?

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import umap

In [ ]:
num_clusters = 20
km = KMeans(n_clusters=num_clusters)
document_embeddings = U * Sigma
km.fit(document_embeddings)
clusters = km.labels_.tolist()
print(clusters)

In [ ]:
embedding = umap.UMAP(n_neighbors=100, min_dist=0.5, random_state=42).fit_transform(document_embeddings)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], c=clusters, s=20, edgecolor='none')
plt.show()

## Problem 2.5 Latent Dirichlet Allocation

Run LDA on `documents` using `sklearn` (find the documentation at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation))

Make sure to specify `random_state=42` for replicability. 

What are the topics allocated to each word of document number 13? 

In [ ]:
print(documents[13])

In [ ]:
# Write your code here.
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification


X, _ = make_multilabel_classification(random_state=0)
lda = LatentDirichletAllocation(n_components=5, random_state=42)

lda.fit(X)